<html>
  <h1  align="center";> Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones
  </h1>
<hr>
  <h3 align="center";>
  Edición 2022
  </h3>
<hr>
  <h1 align="center";>
  AutoML - AutoKeras
  </h1>
</html>


## Integrantes [ Universo Paralelo ]:
- Corral, Diego
- Rinaudo, Eleana
- Romero, Ernest

## Importación de librerías

Importamos la librerías que vamos a utilizar en el práctico.

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    classification_report
)

from curations import encoder_categorical_data_and_scaler_some_data


## Conjuntos de datos

Comenzamos adquiriendo los conjuntos de datos de entrenamiento y prueba del repositorio de GitHub, los cuales utilizamos en el desarrollo del trabajo práctico de la materia obligatoria de Aprendizaje Supervisado. El cual, luego formo parte de la competencia de Kaggle [Spaceship Titanic](https://github.com/eleanarinaudo/Diplo2022_Grupo-5/blob/main/Notebooks/Aprendizaje%20Supervisado/Practico_Spaceship_Titanic.ipynb). 

El objetivo de dicho práctico era el desarrollo de un modelo para dicho conjunto de datos, el cual debía realizar la predicción de cuántos pasajeros fueron transportados a otra dimensión cuando la nave accidentalmente ingresó a la anomalía espacio-temporal. 

A continuación se definen los link a los conjuntos de datos de entrenamiento y prueba.

In [3]:
TITANIC_TRAIN_DATA = "https://raw.githubusercontent.com/eleanarinaudo/AutoML/main/data/titanic_train.csv"
TITANIC_TEST_DATA = "https://raw.githubusercontent.com/eleanarinaudo/AutoML/main/data/titanic_test.csv"

Leemos y generamos los dataset de `train` y `test`, de [Spaceship Titanic](https://github.com/eleanarinaudo/Diplo2022_Grupo-5/blob/main/Notebooks/Aprendizaje%20Supervisado/Practico_Spaceship_Titanic.ipynb) de la competencia de **Kaggle** con su curación respectiva de datos.


In [4]:
train = pd.read_csv(TITANIC_TRAIN_DATA)
test = pd.read_csv(TITANIC_TEST_DATA)


Obervamos el conjunto de datos **train**, y separamos la etiquetas de clasificación "Transported", del conjunto de entrenamiento.

In [5]:
y = train.Transported
X = train.drop("Transported", axis=1)
X.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_of_spaceship,Number_of_cabin,Port_or_Starboard
0,Europa,False,TRAPPIST-1e,1.944636,False,-0.332891,2.099303,-0.266942,-0.097078,3.272797,C,-0.929337,P
1,Mars,True,TRAPPIST-1e,-0.752347,True,-0.332891,-0.281056,-0.288685,-0.269134,-0.270233,F,-0.923471,S
2,Europa,True,TRAPPIST-1e,0.284954,True,-0.332891,-0.281056,-0.288685,-0.269134,-0.270233,B,-0.499119,S
3,Earth,False,TRAPPIST-1e,0.976488,False,-0.123802,0.464079,-0.288685,-0.269134,-0.083662,F,1.606993,P
4,Earth,False,TRAPPIST-1e,-0.683193,False,-0.332891,-0.281056,0.831932,-0.268252,-0.239138,G,-1.152268,P


## Encoder & StandarScaler


Se implementó `LabelEncoder` y `StandarScaler`, los cuales se encuentran implementados en el script `curation.py`. De esta manera, utilizamos StandarScaler para que de este modo los datos sigan una distribución Gaussiana. Para que de esta manera la media sea cercana a 0 y la varianza cercana a 1. Por otro lado, mediante la implementación de `LabelEncoder`, codificamos las columnas categóricas en valores numéricos.

In [6]:
#Encoder & StandarScaler
cat_cols = [
    "HomePlanet",
    "CryoSleep",
    "Destination",
    "VIP",
    "Deck_of_spaceship",
    "Port_or_Starboard",
]

# For XGBoost or Random Forest set the value "general".
model = "general" 
X = encoder_categorical_data_and_scaler_some_data(X, cat_cols=cat_cols, model=model)
X.head()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_of_spaceship,Number_of_cabin,Port_or_Starboard
0,0.433235,-0.737504,0.622444,1.960248,-0.737504,-0.334156,2.118173,-0.268919,-0.096627,3.299609,-1.310594,-0.934458,-1.028924
1,1.689423,1.355926,0.622444,-0.755722,1.355926,-0.334156,-0.282150,-0.290874,-0.270164,-0.271107,0.389513,-0.928546,0.971889
2,0.433235,1.355926,0.622444,0.288882,1.355926,-0.334156,-0.282150,-0.290874,-0.270164,-0.271107,-1.877297,-0.500913,0.971889
3,-0.822954,-0.737504,0.622444,0.985284,-0.737504,-0.123370,0.469234,-0.290874,-0.270164,-0.083078,0.389513,1.621486,-1.028924
4,-0.822954,-0.737504,0.622444,-0.686082,-0.737504,-0.334156,-0.282150,0.840622,-0.269275,-0.239769,0.956215,-1.159113,-1.028924


A continuación realizamos lo mismo sobre el conjunto **test**. 

In [7]:
y_t = test.Transported
X_t = test.drop('Transported',axis=1)
X_t.head()


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_of_spaceship,Number_of_cabin,Port_or_Starboard
0,Europa,True,55 Cancri e,0.382620,False,-0.392550,-0.316125,-0.277604,-0.314802,-0.253534,NaN,-0.875138,NaN
1,Earth,True,PSO J318.5-22,-1.043489,False,-0.392550,-0.316125,-0.277604,-0.314802,-0.253534,G,1.686479,S
2,Earth,False,TRAPPIST-1e,1.061720,False,-0.351821,-0.315467,-0.277604,0.427094,-0.253534,G,0.303656,P
3,Earth,False,PSO J318.5-22,2.352009,False,-0.392550,-0.316125,-0.156820,-0.314802,1.259065,E,-0.082294,S
4,Europa,False,55 Cancri e,-1.247218,False,-0.392550,-0.316125,-0.277604,-0.314802,-0.253534,B,-0.974438,S


In [8]:
X_t = encoder_categorical_data_and_scaler_some_data(X_t, cat_cols=cat_cols, model=model)

In [9]:
X_t.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck_of_spaceship,Number_of_cabin,Port_or_Starboard
0,0.372375,1.184602,-1.836298,0.385213,-0.194446,-0.395332,-0.315904,-0.278164,-0.315288,-0.253794,1.988704,-0.878160,2.729789
1,-0.784647,1.184602,-0.642293,-1.051725,-0.194446,-0.395332,-0.315904,-0.278164,-0.315288,-0.253794,0.868126,1.697716,0.846429
2,-0.784647,-0.699500,0.551713,1.069469,-0.194446,-0.354058,-0.315236,-0.278164,0.434866,-0.253794,0.868126,0.307195,-1.036930
3,-0.784647,-0.699500,-0.642293,2.369555,-0.194446,-0.395332,-0.315904,-0.156089,-0.315288,1.274611,-0.252452,-0.080902,0.846429
4,0.372375,-0.699500,-1.836298,-1.257002,-0.194446,-0.395332,-0.315904,-0.278164,-0.315288,-0.253794,-1.933319,-0.978013,0.846429


----

### XGBOOST

Utilizamos el modelo de `XGBOOST` que fue el mejor resultado obtenido en el práctico de [Spaceship Titanic](https://github.com/eleanarinaudo/Diplo2022_Grupo-5/blob/main/Notebooks/Aprendizaje%20Supervisado/Practico_Spaceship_Titanic.ipynb) con sus correspondientes hiperparámetros.

In [10]:
import xgboost as xgb
from xgboost import XGBClassifier


Para ello, declaramos el modelo XGBClassifier, definimos los hiperparámetros. Posteriormente entrenamos el modelo y realizamos la predicción con el conjunto de test evaluando las métricas obtenidas.

In [11]:
modelXG = xgb.XGBClassifier(objective= 'binary:hinge',
                            n_estimators= 21,
                            max_leaves= 60,
                            max_depth= 6,
                            random_state = 1,
                            tree_method= 'approx'
                           )
modelXG.fit(X, y)

preds_xgb_train = modelXG.predict(X)
preds_xgb_test = modelXG.predict(X_t)

print(classification_report(y, preds_xgb_train))



              precision    recall  f1-score   support

           0       0.87      0.70      0.78      3859
           1       0.76      0.90      0.82      3964

    accuracy                           0.80      7823
   macro avg       0.81      0.80      0.80      7823
weighted avg       0.81      0.80      0.80      7823



In [12]:
accuracy_xgb_train = float(np.sum(preds_xgb_train == y))/y.shape[0]
print('Train Accuracy de XGBoost: ', accuracy_xgb_train)

accuracy_xgb_test = float(np.sum(preds_xgb_test == y_t))/y_t.shape[0]
print('Test Accuracy de XGBoost: ', accuracy_xgb_test)


Train Accuracy de XGBoost:  0.8013549789083472
Test Accuracy de XGBoost:  0.8183908045977012


In [13]:
print(accuracy_score(modelXG.predict(X_t),y_t))

0.8183908045977012


----


## Modelo con AutoML

### AutoKeras

El modelo que elegimos para **AutoML** es `Auto-Keras`.


En nuestro trabajo, vamos ha utilizar `StructuredDataClassifier`. Esta herramienta tiene un parámetro llamado `max_trials` que es responsable del número máximo de modelos Keras diferentes para probar. La búsqueda puede terminar antes de llegar a `max_trials`, siendo el valor predeterminado 50. Si el número es mayor, se tarda más.

Instalamos las librerías necesarias

In [14]:
!pip install autokeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 162 kB 14.6 MB/s 
     |████████████████████████████████| 135 kB 68.9 MB/s 
     |████████████████████████████████| 1.6 MB 66.6 MB/s 


In [15]:
import autokeras as ak


Definimos la cantidad de `epochs=30` y el `max_trial=50`

In [16]:
modelAK = ak.StructuredDataClassifier(overwrite=True, max_trials=50)
modelAK.fit(X,y,epochs=30)


Trial 50 Complete [00h 00m 53s]
val_accuracy: 0.8014184236526489

Best val_accuracy So Far: 0.8214055299758911
Total elapsed time: 00h 34m 44s
Epoch 1/30
245/245 [==============================] - 2s 5ms/step - loss: 0.5358 - accuracy: 0.7264
Epoch 2/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4653 - accuracy: 0.7752
Epoch 3/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4486 - accuracy: 0.7860
Epoch 4/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4386 - accuracy: 0.7895
Epoch 5/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4319 - accuracy: 0.7915
Epoch 6/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4263 - accuracy: 0.7920
Epoch 7/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4216 - accuracy: 0.7950
Epoch 8/30
245/245 [==============================] - 1s 5ms/step - loss: 0.4170 - accuracy: 0.7984
Epoch 9/30
245/245 [==============================] - 1s 

In [17]:
print(accuracy_score(modelAK.predict(X_t),y_t))


28/28 [==============================] - 0s 3ms/step
0.6724137931034483


Comparando con los resultados obtenidos de `XGBOOST`, no obtuvimos mejores resultados. 

---

### FLAML

Otro modelo de AutoML que nos resultó tambien interesante en analizar es el de [FLAML](https://microsoft.github.io/FLAML/).

**FLAML**(Fast and Lightweight AutoML Library) es una librería liviana de Python de código abierto que ayuda a encontrar el mejor modelo de Machine Learning de forma automática, eficiente y económica. Es rápido lo que ahorra tiempo y también tiene un diseño liviano.

Instalamos la librería

In [18]:
!pip install flaml


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 208 kB 14.5 MB/s 
     |████████████████████████████████| 2.0 MB 69.2 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [19]:
from flaml import AutoML

automl_xgb = AutoML()
automl_lgbm = AutoML()


Usamos el estimador `xgboost`

In [20]:
automl_xgb.fit(X, y, task="classification", estimator_list= ['xgboost'] ,time_budget=300)


[flaml.automl: 12-12 03:29:54] {2599} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 12-12 03:29:54] {2601} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 12-12 03:29:54] {2604} INFO - Evaluation method: cv
INFO:flaml.automl:Evaluation method: cv
[flaml.automl: 12-12 03:29:54] {2726} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 12-12 03:29:54] {2870} INFO - List of ML learners in AutoML Run: ['xgboost']
INFO:flaml.automl:List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-12 03:29:54] {3166} INFO - iteration 0, current learner xgboost
INFO:flaml.automl:iteration 0, current learner xgboost
[flaml.automl: 12-12 03:29:54] {3296} INFO - Estimated sufficient time budget=2717s. Estimated necessary time budget=3s.
INFO:flaml.automl:Estimated sufficient time budget=2717s. Estimated necessary time budget=3s.
[flaml.automl: 12-12 

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y, automl_xgb.predict(X)))


              precision    recall  f1-score   support

           0       0.88      0.89      0.89      3859
           1       0.89      0.89      0.89      3964

    accuracy                           0.89      7823
   macro avg       0.89      0.89      0.89      7823
weighted avg       0.89      0.89      0.89      7823



#### XGBOOST con los mejores hiperparámetro de FLAML

Usamos los mejores ajustes de hiperparámetros de FLAML.

In [22]:
modelXG_op = xgb.XGBClassifier(colsample_bylevel=0.7625168189655872,
              colsample_bytree=0.8396884351280078, grow_policy='lossguide',
              learning_rate=0.1346669158664191, max_depth=0, max_leaves=1455,
              min_child_weight=0.42005401172229634, n_estimators=57, n_jobs=-1,
              reg_alpha=3.9409024393567353, reg_lambda=2.9206579909813,
              subsample=1.0, tree_method='hist', use_label_encoder=False,
              verbosity=0)
modelXG_op.fit(X, y)



XGBClassifier(colsample_bylevel=0.7625168189655872,
              colsample_bytree=0.8396884351280078, grow_policy='lossguide',
              learning_rate=0.1346669158664191, max_depth=0, max_leaves=1455,
              min_child_weight=0.42005401172229634, n_estimators=57, n_jobs=-1,
              reg_alpha=3.9409024393567353, reg_lambda=2.9206579909813,
              subsample=1.0, tree_method='hist', use_label_encoder=False,
              verbosity=0)

In [23]:
print(accuracy_score(modelXG_op.predict(X_t),y_t))


0.8160919540229885


#### LightGBM

[LightGBM](https://github.com/microsoft/LightGBM) is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient with the following advantages:

- Faster training speed and higher efficiency.
- Lower memory usage.
- Better accuracy.
- Support of parallel, distributed, and GPU learning.
- Capable of handling large-scale data.



Usamos el estimador `lgbm` (LightGBM)

In [24]:
automl_lgbm.fit(X, y, task="classification", estimator_list= ['lgbm'] ,time_budget=300)

[flaml.automl: 12-12 03:39:10] {2599} INFO - task = classification
INFO:flaml.automl:task = classification
[flaml.automl: 12-12 03:39:10] {2601} INFO - Data split method: stratified
INFO:flaml.automl:Data split method: stratified
[flaml.automl: 12-12 03:39:10] {2604} INFO - Evaluation method: cv
INFO:flaml.automl:Evaluation method: cv
[flaml.automl: 12-12 03:39:10] {2726} INFO - Minimizing error metric: 1-roc_auc
INFO:flaml.automl:Minimizing error metric: 1-roc_auc
[flaml.automl: 12-12 03:39:10] {2870} INFO - List of ML learners in AutoML Run: ['lgbm']
INFO:flaml.automl:List of ML learners in AutoML Run: ['lgbm']
[flaml.automl: 12-12 03:39:10] {3166} INFO - iteration 0, current learner lgbm
INFO:flaml.automl:iteration 0, current learner lgbm
[flaml.automl: 12-12 03:39:10] {3296} INFO - Estimated sufficient time budget=1125s. Estimated necessary time budget=1s.
INFO:flaml.automl:Estimated sufficient time budget=1125s. Estimated necessary time budget=1s.
[flaml.automl: 12-12 03:39:10] {3

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y, automl_lgbm.predict(X)))


              precision    recall  f1-score   support

           0       0.86      0.84      0.85      3859
           1       0.85      0.87      0.86      3964

    accuracy                           0.85      7823
   macro avg       0.85      0.85      0.85      7823
weighted avg       0.85      0.85      0.85      7823



Importamos las librerías

In [26]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

Utilizamos el ajuste de hiperparámetros recomendado por **FLAML**

In [27]:
modelLGBM = LGBMClassifier(colsample_bytree=0.7900822032030472,
               learning_rate=0.12757200414293945, max_bin=127,
               min_child_samples=3, n_estimators=232, num_leaves=8,
               reg_alpha=0.009292081736101283, reg_lambda=0.003866698637831586,
               verbose=-1)
modelLGBM.fit(X, y)



LGBMClassifier(colsample_bytree=0.7900822032030472,
               learning_rate=0.12757200414293945, max_bin=127,
               min_child_samples=3, n_estimators=232, num_leaves=8,
               reg_alpha=0.009292081736101283, reg_lambda=0.003866698637831586,
               verbose=-1)

In [28]:
print(accuracy_score(modelLGBM.predict(X_t),y_t))


0.771264367816092


---

Comparación de los mejores `accuracy_score` de los modelos

In [29]:
report = pd.DataFrame({
    "Model": ["XG Boost",
              "XG Boost con FLAML",
              "Auto Keras",
              "LGBM"
              ],
    "Accuracy score": [accuracy_score(modelXG.predict(X_t),y_t),
                       accuracy_score(modelXG_op.predict(X_t), y_t),
                       accuracy_score(modelAK.predict(X_t), y_t),
                       accuracy_score(modelLGBM.predict(X_t), y_t)
                       ]
})
report.sort_values(by="Accuracy score", ascending=False)


28/28 [==============================] - 0s 3ms/step


,Model,Accuracy score
0,XG Boost,0.818391
1,XG Boost con FLAML,0.816092
3,LGBM,0.771264
2,Auto Keras,0.672414


---
**Conclusión**

Para el práctico, se evaluaron los siguientes modelos: 
- XGBoost con hiperparametros personalizados del práctico anterior del práctico de Aprendizaje Supervisado, 
- XGBoost optimizado con los mejores hiperparámetros obtenidos con FLAML, 
- AutoKeras, 
- LightGBM con los mejores hiperparámetros obtenidos con FLAML. 

Para el caso del modelo XGBoost, obtuvimos scores muy similares entre el conjunto de train y test. Con dicho modelo XGBoost, obtuvimos los mejores valores de accuracy entre todos los modelos analizados. 

Utilizamos el modelo AutoKeras de AutoML, obteniendo el valor más bajo de accuracy como se puede observar en el reporte donde se detalla el score de cada modelo.

En el análsis de modelos, pudimos observar que para el modelo de AutoKeras, está más orientado a problemáticas de deep learning y a dataset con una menor curación previa de datos y con una elevada cantidad de datos. Por otro lado, observamos que incrementando la cantidad de épocas, como así también de iteraciones, se puede lograr un mayor accuracy para el conjunto de datos.

Utilizamos otro modelo de AutoML FLAML, para realizar una comparativa de los hiperparámetros y de esta manera poder evaluar si estos hiperparámetros encontrados mejoran el accuracy del modelo XGBoost previamente entrenado. También se utilizó el modelo LightGBM y se entrenó con los mejores hiperparámetros obtenidos a través de FLAML. Pero, aun así el accuracy obtenidos con estos modelos no superaron al accuracy obtenido con el primer modelo XGBoost entrenado.

Como conclusión final, hemos observado que Autokeras no fue el mejor modelo para la predicción de nuestro Dataset.